###### Investigate 1566 [unassigned] in response_correctness

###### Cross-Tabulation of assignment_attempt_number vs response_correctness

In [ ]:
dfPd = dfInv.toPandas()
# Return cross-tabulation table of assignment_attempt_number vs response_correctness adding counts for null values
pd.crosstab(dfPd.assignment_attempt_number.fillna('null'), dfPd.response_correctness.fillna('null'), margins=True, margins_name="Total")

- 1566 response_correctness "[unassigned]" is the same observations as 1566 assignment_attempt_number and assignment_max_attempts
- investigate additonal correlations

###### Variables Perfectly Correlated to response_correctness = "[unassigned]"

In [ ]:
dfUnassigned = dfInv.filter(F.col("response_correctness") == "[unassigned]")
cols = single_val(dfUnassigned, "response_correctness")

dfUnassigned.select(*cols[:5]).show(1)
dfUnassigned.select(*cols[5:9]).show(1)
dfUnassigned.select(*cols[9:]).show(1)

- each observation is
  - fully scored (learner_attempt_status = "fully scored")
  - same organization
  - has a score (unweighted_final_score varries, not included above)
  - assignment_attempt_number = 0 and assignment_max_attempts = 0 perfectly correlated

###### Date Ranges for response_correctness = "[unassigned]"

In [ ]:
for f in shared['intervalVars']:
  if f not in ["assignment_final_submission_date", "assignment_start_date", "assignment_due_date"]:
    print (f)
    dfUnassigned.agg(
      F.countDistinct(f).alias("unique"),
      F.count(F.when(F.col(f).isNull(), f)).alias("null"),
      F.min(f).alias("min"),
      F.max(f).alias("max")
   ).show(1, False)

- dates spread among school year (9/10/2019 to 4/25/2020)
- max_student_start_datetime and max_student_stop_datetime have the same number of unique values (14)
  - these are entered by teacher
- remaining fields have the same number of unique values (161)

###### Categorical / Identifier Values for response_correctness = "[unassigned]"

In [ ]:
for f in shared['identifierVars']:
  print(f)
  dfUnassigned.agg(
    F.countDistinct(f).alias("unique"),
    F.count(F.when(F.col(f).isNull(), f)).alias("null")
  ).show()

- all scores for one organization
  - 1 organization
  - 5 sections
  - 12 assessments
  - 14 assessment intances
    - 2 repeated
    - see datetime variables above with 161 values
  - 64 learners
  - 161 attempts
    - see datetime variables above with 161 values
  - 1566 questions

####### Conclusion: 1566 [unassigned] in response_correctness
- isolated to one organization
- no pattern, an outlier
- 1566 is 2% of observations
  - 1566 / 80548
- remove 1566 rows
- binary variables assignment_attempt_number and assignment_max_attempts
  - will be unary after 1566 deleted
  - remove variables

In [ ]:
finish_todo("Investigate 1566 [unassigned] in response_correctness")
finish_todo('Investigate assignment_attempt_number and assignment_max_attempts both have 1566 values')

###### Remove 1566 [unassigned] in response_correctness observations"

In [ ]:
# Make sure to preserve the nulls
dfInv = dfInv.filter(F.col("response_correctness").isNull() | (F.col("response_correctness") != "[unassigned]"))
